<a href="https://colab.research.google.com/github/zzong2006/predict_churn_risk_rate/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>